# HW05 : Text Classification

ในหัวข้อนี้จะได้เรียนรู้การสร้างโมเดลสำหรับจำแนกข้อความ โดยใช้วิธีต่างๆ ดังนี้

1) Naïve Bayes

2) Classical ML : TF-IDF (used as input features)


# Data Preparation

ในบทนี้เราจะใช้ข้อมูลของ Wongnai Sentiment Corpus มาใช้ในการสร้างแบบจำลอง



In [1]:
#download data
!wget https://raw.githubusercontent.com/PyThaiNLP/wisesight-sentiment/master/pos.txt
!wget https://raw.githubusercontent.com/PyThaiNLP/wisesight-sentiment/master/neg.txt

#create train/validate/test datasets
from math import floor

def split_train_test_sets(file_list,train_num=0.8,val_num=0.1,test_num=0.1):

    split_train_index = floor(len(file_list) * train_num)
    split_val_index = floor(len(file_list) * (train_num+val_num))
    train = file_list[:split_train_index]
    val = file_list[split_train_index:split_val_index]
    test = file_list[split_val_index:]
    return train, val, test

def file2list(fname):
  with open(fname) as fp:
    lines = fp.readlines()
    return [line.strip() for line in lines]
def list2file(items,fname):
  file = open(fname,'w')
  for item in items:
    file.write(item.strip()+"\n")
  file.close()

pos_text = file2list("pos.txt")
neg_text = file2list("neg.txt")

--2023-12-22 09:27:39--  https://raw.githubusercontent.com/PyThaiNLP/wisesight-sentiment/master/pos.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 761195 (743K) [text/plain]
Saving to: ‘pos.txt’

pos.txt             100%[===================>] 743.35K  4.80MB/s    in 0.2s    

2023-12-22 09:27:40 (4.80 MB/s) - ‘pos.txt’ saved [761195/761195]

--2023-12-22 09:27:40--  https://raw.githubusercontent.com/PyThaiNLP/wisesight-sentiment/master/neg.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1676577 (1.6M) [text/plain]
Saving 

In [2]:
import pandas as pd
import re
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
import math
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from collections import defaultdict



In [3]:
pos_df = pd.DataFrame(pos_text,columns=['review'])
pos_df['label'] = "positive"

neg_df = pd.DataFrame(neg_text,columns=['review'])
neg_df['label'] = "negative"
neg_df

data = pd.concat([pos_df, neg_df])

data.sample(10)

,review,label
4426,สกุณา โผผิน บินที่หมาย แม้ร่างกาย แรงอ่อน ร่อน...,positive
3218,ตอนแรกก็ชอบหงส์ พอภาษีขึ้น ชอบแสงโสมดีกว่า,positive
30,😣😣😣,negative
6278,แพงจริง ๆ ครับ เจ็ดร้อยที่เขาว่า ได้กุ้งแชบ๊วย...,negative
5523,สมชัย มึงจะออกมาบ่นทำแมวน้ำอะไร มึงเป้นคนเป่าเ...,negative
1434,ได้ค่ะ ยินดีค่ะ,positive
5491,บริการแย่มากกกก ปรับปรุงตรงนี้ดีกว่า ทั้งคอลเซ...,negative
363,> ป่ะเรา,positive
3049,เบน285 แม่โขง แสงโสม รีย์ ก็ดีน่ะ 555,positive
530,หูยยยยยย,positive


In [4]:
category_counts = data['label'].value_counts()

category_counts

negative    6823
positive    4778
Name: label, dtype: int64

# Naïve Bayes


โค้ดส่วนนี้ปรับมาจาก https://www.analyticsvidhya.com/blog/2022/03/building-naive-bayes-classifier-from-scratch-to-perform-sentiment-analysis/


In [5]:
!pip install pythainlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 26.1 MB/s eta 0:00:00


In [6]:

from pythainlp.tokenize import word_tokenize
import re

def deEmojify(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)

def thai_clean_text(text):
    st = ""

    #สามารถเพิ่มโค้ด สำหรับคลีน ข้อความ เช่น ลบ emoji ออก เป็นต้น
    #text = deEmojify(text)
    for w in word_tokenize(text):
        st = st + w + " "
    return  re.sub(' +', ' ', st)

data['review'] = data.review.apply(thai_clean_text)

data

,review,label
0,: 3,positive
1,☺️,positive
2,🤤,positive
3,🤪,positive
4,😁,positive
...,...,...
6818,รางวัล อะไร อินเตอร์ ตาย ล่ะ # True เป็น บริษั...,negative
6819,เป็น อัยการ ภาษาไทย นี่แหละ ครับ อ่าน ครับ ที่...,negative
6820,ข้าว โถ ละ ร้อย แพง เพราะ ตัก เป็น จาน ๆ ละ 15...,negative
6821,ก่อนหน้านี้ เคย พูดถึง สิ่ง ที่ Isuzu D-Max คว...,negative


In [7]:
reviews = data['review'].values
labels = data['label'].values
encoder = LabelEncoder()
encoded_labels = encoder.fit_transform(labels)

encoded_labels

array([1, 1, 1, ..., 0, 0, 0])

In [8]:
print(labels)

['positive' 'positive' 'positive' ... 'negative' 'negative' 'negative']


In [9]:
train_sentences, test_sentences, train_labels, test_labels = train_test_split(reviews, encoded_labels, stratify = encoded_labels)

In [10]:
from tqdm import tqdm
vec = CountVectorizer(max_features = 1000)
X = vec.fit_transform(train_sentences)
vocab = vec.get_feature_names_out()
X = X.toarray()
word_counts = {}

#ใช้ข้อมูล  2 class
for l in range(2):
    word_counts[l] = defaultdict(lambda: 0)
for i in tqdm(range(X.shape[0])):
    l = train_labels[i]
    for j in range(len(vocab)):
        word_counts[l][vocab[j]] += X[i][j]

100%|██████████| 8700/8700 [00:15<00:00, 552.84it/s]


In [11]:
def laplace_smoothing(n_label_items, vocab, word_counts, word, text_label):
    a = word_counts[text_label][word] + 1
    b = n_label_items[text_label] + len(vocab)
    return math.log(a/b)

def group_by_label(x, y, labels):
    data = {}
    for l in labels:
        data[l] = x[np.where(y == l)]

    #print(data)
    return data

def fit(x, y, labels):
    n_label_items = {}
    log_label_priors = {}
    n = len(x)
    grouped_data = group_by_label(x, y, labels)
    for l, data in grouped_data.items():

        n_label_items[l] = len(data)
        log_label_priors[l] = math.log(n_label_items[l] / n)
    return n_label_items, log_label_priors

def predict(n_label_items, vocab, word_counts, log_label_priors, labels, x):
    result = []
    for text in x:
        label_scores = {l: log_label_priors[l] for l in labels}

        words = set(text.strip().split(" ")) #split text with space
        #print(words)
        for word in words:
            if word not in vocab: continue
            for l in labels:
                log_w_given_l = laplace_smoothing(n_label_items, vocab, word_counts, word, l)
                label_scores[l] += log_w_given_l
        #print(max(label_scores, key=label_scores.get))
        result.append(max(label_scores, key=label_scores.get))
    return result

In [12]:
labels = [0,1]
n_label_items, log_label_priors = fit(train_sentences,train_labels,labels)

In [13]:
pred = predict(n_label_items, vocab, word_counts, log_label_priors, labels, test_sentences)
print("Accuracy of prediction on test set : ", accuracy_score(test_labels,pred))

Accuracy of prediction on test set :  0.6370217166494312


In [14]:
from sklearn.metrics import classification_report
print(classification_report(test_labels,pred))

              precision    recall  f1-score   support

           0       0.62      0.97      0.76      1706
           1       0.81      0.15      0.26      1195

    accuracy                           0.64      2901
   macro avg       0.72      0.56      0.51      2901
weighted avg       0.70      0.64      0.55      2901



# Classical ML + TF-IDF

ในส่วนนี้เราจะใช้ข้อมูลชุดเดิมและใช้ TF-IDF ในการแทนแต่ละประโยคหลังจากนั้นก็เทรน ด้วยโมเดล Machine learning ที่นิยมใช้กันทั่วไป

-  Logistic regression

โค้ดดัดแปลงมาจาก https://colab.research.google.com/drive/1SR_j7FUYcJ21CCsIT8fkRjGOteevQEPa#scrollTo=4Wf_YSr78IMO

In [15]:
reviews = data['review'].values
labels = data['label'].values
encoder = LabelEncoder()
encoded_labels = encoder.fit_transform(labels)

train_sentences, test_sentences, train_labels, test_labels = train_test_split(reviews, encoded_labels, stratify = encoded_labels,test_size=0.2, random_state=1234)

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
import torch

train = list(train_sentences)
test = list(test_sentences)
#STOPWORDS = set(stopwords.words('english'))

tfidf_vectorizer = TfidfVectorizer(max_features=1000, dtype=np.float32)

#ใช้ชุดเทรน ทำ vectorizer
tfidfX_train = tfidf_vectorizer.fit_transform(train)
tfidfX_test = tfidf_vectorizer.transform(test)

print("TF-IDF train shape:", tfidfX_train.shape)
print("TF-IDF test shape:", tfidfX_test.shape)

TF-IDF train shape: (9280, 1000)
TF-IDF test shape: (2321, 1000)


In [17]:
import torch
Y_train = train_labels
Y_val = test_labels
Y_test = test_labels

print("Y train shape:", Y_train.shape)
print("Y test shape:", Y_test.shape)

Y train shape: (9280,)
Y test shape: (2321,)


In [18]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

classifier = LogisticRegression()
classifier.fit(tfidfX_train, Y_train)

LogisticRegression()

In [19]:
y_pred = classifier.predict(tfidfX_test)

In [20]:
#Show the classification report
print(classification_report(Y_test,y_pred))

              precision    recall  f1-score   support

           0       0.83      0.88      0.85      1365
           1       0.81      0.75      0.78       956

    accuracy                           0.82      2321
   macro avg       0.82      0.81      0.82      2321
weighted avg       0.82      0.82      0.82      2321



In [21]:
from sklearn.tree import DecisionTreeClassifier

In [22]:
dt_classifier = DecisionTreeClassifier(random_state=42)
dt_classifier.fit(tfidfX_train, Y_train)

DecisionTreeClassifier(random_state=42)

In [23]:
y_pred = dt_classifier.predict(tfidfX_test)

print(classification_report(Y_test,y_pred))

              precision    recall  f1-score   support

           0       0.78      0.79      0.79      1365
           1       0.70      0.68      0.69       956

    accuracy                           0.75      2321
   macro avg       0.74      0.74      0.74      2321
weighted avg       0.75      0.75      0.75      2321



## Assigment

จากตัวอย่างโค้ดให้นักศึกษา สร้างแบบจำลองในการจำแนกกลุ่มของข้อความมากกว่า 2 คลาส โดยใช้ LogisticRegression และใช้เวกเตอร์ลักษณะสำคัญ TF-IDF

โดยใช้ข้อมูลจาก wisesight-sentiment ซึงจะมีข้อความอยู่ 4 ประเภท

* ข้อความเชิงบวก (Positive)
* ข้อความเชิงลบ (Negative)
* ข้อความปกติ ไม่เป็นเชิงบวกหรือเชิงลบ (Neutral)
* ข้อความลักษณะคำถาม (Question)



In [24]:
#positive
!wget https://raw.githubusercontent.com/PyThaiNLP/wisesight-sentiment/master/pos.txt

#negative
!wget https://raw.githubusercontent.com/PyThaiNLP/wisesight-sentiment/master/neg.txt

#Neutral
!wget https://raw.githubusercontent.com/PyThaiNLP/wisesight-sentiment/master/neu.txt

#Question
!wget https://raw.githubusercontent.com/PyThaiNLP/wisesight-sentiment/master/q.txt

--2023-12-22 09:32:13--  https://raw.githubusercontent.com/PyThaiNLP/wisesight-sentiment/master/pos.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 761195 (743K) [text/plain]
Saving to: ‘pos.txt.1’

pos.txt.1           100%[===================>] 743.35K  4.45MB/s    in 0.2s    

2023-12-22 09:32:13 (4.45 MB/s) - ‘pos.txt.1’ saved [761195/761195]

--2023-12-22 09:32:13--  https://raw.githubusercontent.com/PyThaiNLP/wisesight-sentiment/master/neg.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1676577 (1.6M) [text/plain]
Sav

In [ ]:
#Your code here!

In [25]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report

In [27]:
positive_data = open("pos.txt", encoding="utf-8").readlines()
negative_data = open("neg.txt", encoding="utf-8").readlines()
neutral_data = open("neu.txt", encoding="utf-8").readlines()
question_data = open("q.txt", encoding="utf-8").readlines()

In [28]:
reviews = positive_data + negative_data + neutral_data + question_data
labels = ["positive"] * len(positive_data) + ["negative"] * len(negative_data) + ["neutral"] * len(neutral_data) + ["question"] * len(question_data)

In [29]:
encoder = LabelEncoder()
encoded_labels = encoder.fit_transform(labels)

In [30]:
train_sentences, test_sentences, train_labels, test_labels = train_test_split(reviews, encoded_labels, stratify=encoded_labels, test_size=0.2, random_state=1234)

In [31]:
tfidf_vectorizer = TfidfVectorizer(max_features=1000, dtype=np.float32)
tfidfX_train = tfidf_vectorizer.fit_transform(train_sentences)
tfidfX_test = tfidf_vectorizer.transform(test_sentences)

In [32]:
classifier = LogisticRegression()
classifier.fit(tfidfX_train, train_labels)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [37]:
y_pred_ = classifier.predict(tfidfX_test)

In [38]:
print("Logistic Regression:")
print(classification_report(test_labels, y_pred_))

Logistic Regression:
              precision    recall  f1-score   support

           0       0.62      0.50      0.56      1365
           1       0.63      0.86      0.73      2912
           2       0.56      0.17      0.26       956
           3       0.38      0.03      0.05       115

    accuracy                           0.63      5348
   macro avg       0.55      0.39      0.40      5348
weighted avg       0.61      0.63      0.59      5348



In [39]:
dt_classifier = DecisionTreeClassifier(random_state=42)
dt_classifier.fit(tfidfX_train, train_labels)
dt_y_pred = dt_classifier.predict(tfidfX_test)

In [40]:
print("Decision Tree:")
print(classification_report(test_labels, dt_y_pred))

Decision Tree:
              precision    recall  f1-score   support

           0       0.47      0.47      0.47      1365
           1       0.62      0.72      0.67      2912
           2       0.34      0.19      0.25       956
           3       0.09      0.04      0.06       115

    accuracy                           0.55      5348
   macro avg       0.38      0.36      0.36      5348
weighted avg       0.52      0.55      0.53      5348

